# Path to eQE 2.0

## The problem
embedded Quantum ESPRESSO is a fast and capable software, but it's also written with poor coding standards. Thus developing and extending the codebase is extremely painful and prone to errors (trust him, he wrote it).

The main reason why the situation is so dire, is that eQE is entangled with the Quantum Espresso (QE) code, and QE happens to have huge flaws in the way it is written.

What we believe is the worst problem is that QE is not compliant to <i>functional paradigm</i> standards.

> In computer science, functional programming is a programming paradigm—a style of building the structure and elements of computer programs—that treats computation as the evaluation of mathematical functions and avoids changing-state and mutable data.


#### But what does it mean?

A function provided the same inputs should always return the same output.
Let's see some code:

##### The logical way (eQE 2.0)

In [ ]:
def f0(x,y,l):
    l.append(x)
    l.append(y)
    return l

##### The I-hate-myself way (QE / eQE 1.0)

In [ ]:
l = []

def f1(x,y):
    l.append(x)
    l.append(y)
    return l

##### Runtime

In [ ]:
print( "f0: ", f0(1,2,[]) )
print( "f1: ", f1(1,2) )

## The solution

To improve eQE, it is vital to make QE better.

If the QE base was written in a logical functional way, it would be so much easier to disentangle the special case subsystem approach, and have the two codes live independently.

Since QE developers wouldn't do it, we decided to fix it ourselves.

QE is several million lines long, and adapting it to the functional paradigm by hand not only would be extremely tedious, but it would need to be done over and over for each new major release.

We hate boring, so we'll have the computer do it for us.

### Phase 0: mapping (80% complete)

Generate a script to create a map of the sourcecode.
For each `f90` file we need to know:
- Which modules/function/subroutines are there
- The parent/child relation between blocks of code
- Which are the arguments of a function/routine
- Which quantities are imported from external modules (<strong>the source of the problem</strong>)

In [ ]:
import numpy as np
import codeblocks as cb
import deprecated_use as du
import f90_regex as frgx

%load_ext autoreload
%autoreload 2
files = [] # !ls /home/alessandro/QE/qe-6.0/Modules/*f90
files2 = !ls /home/alessandro/QE/qe-6.0/PW/src/*f90
files = files + files2
#files = ['/home/alessandro/QE/qe-6.0/PW/src/symm_base.f90']
QE = cb.Project('QE')
for file in files:
    QE.add_sourcefile(file)
QE.scan_sources()
QE.make_project_tree()

#### Information gathered during mapping

Let's take the subroutine where the Hartree potential is computed, `v_h`, for example.

```fortran
!----------------------------------------------------------------------------
SUBROUTINE v_h( rhog, ehart, charge, v )
  !----------------------------------------------------------------------------
  !
  ! ... Hartree potential VH(r) from n(G)
  !
  USE constants, ONLY : fpi, e2
  USE kinds,     ONLY : DP
  USE fft_base,  ONLY : dfftp
  USE fft_interfaces,ONLY : invfft
  USE gvect,     ONLY : nl, nlm, ngm, gg, gstart
  USE lsda_mod,  ONLY : nspin
  USE cell_base, ONLY : omega, tpiba2
  USE control_flags, ONLY : gamma_only
  USE mp_bands,  ONLY: intra_bgrp_comm
  USE mp,        ONLY: mp_sum
  USE martyna_tuckerman, ONLY : wg_corr_h, do_comp_mt
  USE esm,       ONLY: do_comp_esm, esm_hartree, esm_bc

```

In [ ]:
for file in QE.sources:
    if file.name.endswith('v_of_rho.f90'): 
        v_h = file.contains['v_h']
        break

v_h.imports

In [ ]:
from deprecated_use import depr
print("Deprecated master list:",depr)

In [ ]:
target_subroutine = v_h
deprecated = {}

for module, quantities in depr.items():
    if module in target_subroutine.imports :
        deprecated[module] = []
        if quantities is None:
            deprecated[module] = target_subroutine.imports[module]
        else:
            if target_subroutine.imports[module] is None:
                deprecated[module] = None
            else:
                for quantity in target_subroutine.imports[module]:
                    if quantity in depr[module]:
                        deprecated[module].append(quantity)

print("Deprecated quantities found in codeblock:", deprecated)

### Phase 1: Designing DataTypes (pre-hackathon?)

- We will create a series of datatypes that mimic the structure of existing deprecated modules.
- Attributes inside these datatypes will be pointers to the actual quantities in the modules
- The optimal way would be to create class instances instead of duplicated static modules, but hey, we want to leave some of the work to the actual QE developers, right?
- There will be module duplication, but by passing around the datatypes we will avoid routine/function duplication, which is a way more pressing issue.

### Phase 2: Patching (hackathon)

Now that we have a map of where the problems are, the script will need to be extended to:
- Fix the argument lists of each function/subroutine to include passing of the datatypes
- Remove the toxic `USE` statements
- Assign pointers within the routines so that the names in the code later on doesn't need to be changed
- Fix the routines calls to match the new argument scheme.


### Phase 3...N: Writing eQE 2.0 (the distant future)